In [1]:
%cd are-gnn-defenses-robust/

[Errno 2] No such file or directory: 'are-gnn-defenses-robust/'
/home/niyati/ERSP/2-phase-attack/experiments/2-phase


/home/niyati/miniconda3/envs/ersp_v2/lib/python3.9/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [2]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy

sys.path.append('../../')

from src.models.gcn import *
from src.utils.datasets import *
from src.models.trainable import *

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset_directory = "../Cora"
cora_dataset = Planetoid(root=dataset_directory, name='Cora')
data = cora_dataset[0].to(device)
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [5]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [6]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 500)

Epoch 0, Train Loss - 4.12787389755249, Val Loss - 2.775822639465332, Val Accuracy - 0.198
Epoch 20, Train Loss - 0.2716486155986786, Val Loss - 1.255977749824524, Val Accuracy - 0.719
Epoch 40, Train Loss - 0.1236221194267273, Val Loss - 1.8354213237762451, Val Accuracy - 0.697
Epoch 60, Train Loss - 0.07564207166433334, Val Loss - 2.101832628250122, Val Accuracy - 0.705
Epoch 80, Train Loss - 0.05289803817868233, Val Loss - 2.2457163333892822, Val Accuracy - 0.713
Epoch 100, Train Loss - 0.06918197125196457, Val Loss - 2.128267288208008, Val Accuracy - 0.712
Epoch 120, Train Loss - 0.04970696568489075, Val Loss - 2.200208902359009, Val Accuracy - 0.727
Epoch 140, Train Loss - 0.03290263190865517, Val Loss - 2.3902268409729004, Val Accuracy - 0.711
Epoch 160, Train Loss - 0.09823440760374069, Val Loss - 2.6008646488189697, Val Accuracy - 0.711
Epoch 180, Train Loss - 0.026149509474635124, Val Loss - 2.372742176055908, Val Accuracy - 0.721
Epoch 200, Train Loss - 0.02600572444498539, V

In [9]:
import sys
import torch
from itertools import count
from torch_geometric.utils import dense_to_sparse, to_networkx, from_networkx
import random
import numpy as np
import networkx as nx


# Get initial accuracy
_, initial_accuracy = train.test(data)


# Convert graph to NetworkX for manipulation
G = to_networkx(data, to_undirected=True)

# Identify nodes in the same class
class_nodes = {}  # Dictionary to group nodes by class
for node, label in enumerate(data.y.tolist()):
    if label not in class_nodes:
        class_nodes[label] = []
    class_nodes[label].append(node)

# Generate potential edges within each class
edges_to_add = []
num_edges_to_add = 100  # Define the number of edges to add

for label, nodes in class_nodes.items():
    if len(nodes) > 1:
        # Randomly pair nodes within the same class
        random.shuffle(nodes)
        for i in range(len(nodes)):
            for j in range(i + 1, len(nodes)):
                if not G.has_edge(nodes[i], nodes[j]):
                    edges_to_add.append((nodes[i], nodes[j]))

# Randomly select a subset of edges to add
random.shuffle(edges_to_add)
edges_to_add = edges_to_add[:num_edges_to_add]

# Add the selected edges to the graph
for i, j in edges_to_add:
    G.add_edge(i, j)

# Convert back to PyTorch Geometric format
modified_data = from_networkx(G).to(device)
modified_data.x = data.x  # Preserve original node features
modified_data.y = data.y  # Preserve original labels
modified_data.train_mask = data.train_mask
modified_data.test_mask = data.test_mask

# Evaluate the model on the modified graph
print("Training on modified graph...")
train.model.reset_parameters()  # Reset model parameters
train.fit(modified_data, 500)

# Get modified accuracy
_, modified_accuracy = train.test(modified_data)
print(f"Modified Accuracy: {modified_accuracy}")

# Output accuracy change
accuracy_change = modified_accuracy - initial_accuracy
print(f"Accuracy Change: {accuracy_change:.4f}")

# Calculate and display edge statistics
initial_edges = G.number_of_edges() - len(edges_to_add)
final_edges = G.number_of_edges()
print(f"Edges Added: {final_edges - initial_edges}")
print(f"Percentage Increase in Edges: {100 * (final_edges - initial_edges) / initial_edges:.2f}%")

Training on modified graph...
Epoch 0, Train Loss - 5.169620513916016, Val Loss - 5.614458084106445, Val Accuracy - 0.154
Epoch 20, Train Loss - 0.562423050403595, Val Loss - 1.6728888750076294, Val Accuracy - 0.755
Epoch 40, Train Loss - 0.24332641065120697, Val Loss - 2.2070152759552, Val Accuracy - 0.729
Epoch 60, Train Loss - 0.2585086226463318, Val Loss - 2.177323341369629, Val Accuracy - 0.721
Epoch 80, Train Loss - 0.18270914256572723, Val Loss - 2.197629451751709, Val Accuracy - 0.723
Epoch 100, Train Loss - 0.16466465592384338, Val Loss - 2.2891266345977783, Val Accuracy - 0.721
Epoch 120, Train Loss - 0.1419861614704132, Val Loss - 2.3344905376434326, Val Accuracy - 0.723
Epoch 140, Train Loss - 0.13569709658622742, Val Loss - 2.4149649143218994, Val Accuracy - 0.73
Epoch 160, Train Loss - 0.11069643497467041, Val Loss - 2.3237829208374023, Val Accuracy - 0.739
Epoch 180, Train Loss - 0.1025346964597702, Val Loss - 2.31205153465271, Val Accuracy - 0.736
Epoch 200, Train Loss 

In [10]:
# Convert graph to NetworkX for manipulation
G = to_networkx(data, to_undirected=True)

# Identify nodes in the same class
class_nodes = {}  # Dictionary to group nodes by class
for node, label in enumerate(data.y.tolist()):
    if label not in class_nodes:
        class_nodes[label] = []
    class_nodes[label].append(node)

# Add edges between same-class nodes if they don't already exist
edges_added = 0  # Counter for edges added
for label, nodes in class_nodes.items():
    for i, source in enumerate(nodes):
        for target in nodes[i + 1:]:  # Avoid self-loops and duplicate edges
            if not G.has_edge(source, target):
                G.add_edge(source, target)
                edges_added += 1

# Convert back to PyTorch Geometric format
modified_data = from_networkx(G).to(device)
modified_data.x = data.x  # Preserve original node features
modified_data.y = data.y  # Preserve original labels
modified_data.train_mask = data.train_mask
modified_data.test_mask = data.test_mask

# Evaluate the model on the modified graph
print("Training on modified graph...")
train.model.reset_parameters()  # Reset model parameters
train.fit(modified_data, 500)

# Get modified accuracy
_, modified_accuracy = train.test(modified_data)
print(f"Modified Accuracy: {modified_accuracy}")

# Output accuracy change
accuracy_change = modified_accuracy - initial_accuracy
print(f"Accuracy Change: {accuracy_change:.4f}")

# Display edge statistics
initial_edges = G.number_of_edges() - edges_added
final_edges = G.number_of_edges()
print(f"Edges Added: {edges_added}")
print(f"Percentage Increase in Edges: {100 * edges_added / initial_edges:.2f}%")

Training on modified graph...
Epoch 0, Train Loss - 11263.3798828125, Val Loss - 76011.5390625, Val Accuracy - 0.144
Epoch 20, Train Loss - 551.4589233398438, Val Loss - 226.26904296875, Val Accuracy - 0.846
Epoch 40, Train Loss - 26.256793975830078, Val Loss - 0.5905293226242065, Val Accuracy - 0.999
Epoch 60, Train Loss - 0.0, Val Loss - 0.8070293068885803, Val Accuracy - 0.998
Epoch 80, Train Loss - 0.0, Val Loss - 0.0, Val Accuracy - 1.0
Epoch 100, Train Loss - 0.0, Val Loss - 0.0, Val Accuracy - 1.0
Epoch 120, Train Loss - 0.0, Val Loss - 0.0, Val Accuracy - 1.0
Epoch 140, Train Loss - 0.0, Val Loss - 0.8717910051345825, Val Accuracy - 0.999
Epoch 160, Train Loss - 0.0, Val Loss - 0.34303125739097595, Val Accuracy - 0.999
Epoch 180, Train Loss - 0.0, Val Loss - 0.27806249260902405, Val Accuracy - 0.999
Epoch 200, Train Loss - 0.0, Val Loss - 0.27000001072883606, Val Accuracy - 0.999
Epoch 220, Train Loss - 0.0, Val Loss - 0.2690175771713257, Val Accuracy - 0.999
Epoch 240, Train L